<a href="https://colab.research.google.com/github/Namumbya/Namumbya.github.io/blob/main/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np

In [10]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = []
        self.biases = []
        for i in range(1, len(layers)):
            self.weights.append(np.random.randn(layers[i], layers[i-1]))
            self.biases.append(np.random.randn(layers[i], 1))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def forward_propagation(self, X):
        a = X
        activations = [a]
        zs = []
        for i in range(len(self.layers)-1):
            z = np.dot(self.weights[i], a) + self.biases[i]
            zs.append(z)
            a = self.sigmoid(z)
            activations.append(a)
        return activations, zs

    def backpropagation(self, X, y):
        m = X.shape[1]
        activations, zs = self.forward_propagation(X)
        delta_weights = [np.zeros(w.shape) for w in self.weights]
        delta_biases = [np.zeros(b.shape) for b in self.biases]

        delta = (activations[-1] - y) * self.sigmoid_derivative(zs[-1])
        delta_weights[-1] = np.dot(delta, activations[-2].T)
        delta_biases[-1] = np.sum(delta, axis=1, keepdims=True)

        for l in range(2, len(self.layers)):
            z = zs[-l]
            sp = self.sigmoid_derivative(z)
            delta = np.dot(self.weights[-l+1].T, delta) * sp
            delta_weights[-l] = np.dot(delta, activations[-l-1].T)
            delta_biases[-l] = np.sum(delta, axis=1, keepdims=True)

        return delta_weights, delta_biases

    def train(self, X, y, epochs, learning_rate):
        m = X.shape[1]
        for epoch in range(epochs):
            delta_weights = [np.zeros(w.shape) for w in self.weights]
            delta_biases = [np.zeros(b.shape) for b in self.biases]
            for i in range(m):
                x = X[:, i].reshape(-1, 1)
                y_i = y[:, i].reshape(-1, 1)
                dw, db = self.backpropagation(x, y_i)
                delta_weights = [dw_l + dw_l_i for dw_l, dw_l_i in zip(delta_weights, dw)]
                delta_biases = [db_l + db_l_i for db_l, db_l_i in zip(delta_biases, db)]
            self.weights = [w - (learning_rate / m) * dw for w, dw in zip(self.weights, delta_weights)]
            self.biases = [b - (learning_rate / m) * db for b, db in zip(self.biases, delta_biases)]

    def predict(self, X):
        X = X.T
        activations, _ = self.forward_propagation(X)
        return activations[-1].T

In [11]:
# Example usage:
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
y = np.array([[0, 1, 1, 0]])
layers = [2, 4, 1]  # 2 input units, 4 hidden units, 1 output unit

In [12]:
nn = NeuralNetwork(layers)
nn.train(X, y, epochs=10, learning_rate=0.1)

In [13]:
# Test the trained neural network
test_input = np.array([[1, 0]])
predicted_output = nn.predict(test_input)
print("Predicted output:", predicted_output)

Predicted output: [[0.50023055]]
